## Scraping data

In [29]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [30]:
# Scrape from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_of_Los_Angeles'

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [31]:
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="div-col columns column-width")[0].findAll("li"):
    neighborhoodList.append(row.text)
    

In [88]:
import re 
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list
def remove(list): 
    list = [re.sub(r'\[.*?\]', '', i) for i in list] 
    return list

list = neighborhoodList
cleanlist = remove(list) 

cleanlist


['Angelino Heights',
 'Angeles Mesa',
 'Angelus Vista',
 'Arleta',
 'Arlington Heights',
 'Arts District',
 'Atwater Village',
 'Baldwin Hills',
 'Baldwin Hills/Crenshaw',
 'Baldwin Village',
 'Baldwin Vista',
 'Beachwood Canyon',
 'Bel Air, Bel-Air or Bel Air Estates',
 'Benedict Canyon',
 'Beverly Crest',
 'Beverly Glen',
 'Beverly Grove',
 'Beverly Hills Post Office',
 'Beverly Park',
 'Beverlywood',
 'Boyle Heights',
 'Brentwood',
 'Brentwood Circle',
 'Brentwood Glen',
 'Broadway-Manchester',
 'Brookside',
 'Bunker Hill',
 'Cahuenga Pass',
 'Canoga Park',
 'Canterbury Knolls',
 'Carthay',
 'Castle Heights',
 'Central-Alameda',
 'Central City',
 'Century City',
 'Chatsworth',
 'Chesterfield Square',
 'Cheviot Hills',
 'Chinatown',
 'Civic Center',
 'Crenshaw',
 'Crestwood Hills',
 'Cypress Park',
 'Del Rey',
 'Downtown',
 'Eagle Rock',
 'East Gate Bel Air',
 'East Hollywood',
 ' East Los Angeles',
 'Echo Park',
 'Edendale',
 'El Sereno',
 'Elysian Heights',
 'Elysian Park',
 'Elysi

In [89]:
la_df = pd.DataFrame({"Neighborhood": cleanlist})
la_df.head()

,Neighborhood
0,Angelino Heights
1,Angeles Mesa
2,Angelus Vista
3,Arleta
4,Arlington Heights


## Matching coordinates

In [97]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 8.5MB/s ta 0:00:011


In [98]:

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [101]:
coords = [ get_latlng(neighborhood) for neighborhood in la_df["Neighborhood"].tolist() ]

In [102]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [103]:
# merge the coordinates into the original dataframe
la_df['Latitude'] = df_coords['Latitude']
la_df['Longitude'] = df_coords['Longitude']
la_df.head()

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070290,-118.254800
1,Angeles Mesa,18.285830,-66.800890
2,Angelus Vista,34.087575,-118.267156
3,Arleta,34.249050,-118.433490
4,Arlington Heights,34.039890,-118.325420


## Create map

In [105]:
#import packages
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip -q install folium
import folium

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [106]:
from geopy.geocoders import Nominatim 

address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [111]:
# create map
map_la = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(la_df['Latitude'], la_df['Longitude'], la_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_la)  
    
map_la

## Explore neighbourhood

In [112]:
# foursquare API
CLIENT_ID = 'O3MIXLO0F1UQFN3RFCXG2WOASJNB0WMBYA2UPOC0PZ20GA1F' 
CLIENT_SECRET = 'GLPHADBOLKCYLMKNC4NBDZM1FSR2UI0EO4CGT4GPX5SZYYGK' 
VERSION = '20180605' 

In [114]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(la_df['Latitude'], la_df['Longitude'], la_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [115]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8170, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Angelino Heights,34.07029,-118.2548,Guisados,34.070262,-118.250437,Taco Place
1,Angelino Heights,34.07029,-118.2548,Eightfold Coffee,34.071245,-118.250698,Coffee Shop
2,Angelino Heights,34.07029,-118.2548,Halliwell Manor,34.069329,-118.254165,Performing Arts Venue
3,Angelino Heights,34.07029,-118.2548,Ototo,34.072659,-118.251740,Sake Bar
4,Angelino Heights,34.07029,-118.2548,Subliminal Projects,34.072290,-118.250737,Art Gallery


In [117]:
# check number of venues
venues_df.groupby(["Neighborhood"]).count()
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 365 uniques categories.


In [118]:
# show unique venue types

venues_df['VenueCategory'].unique()

array(['Taco Place', 'Coffee Shop', 'Performing Arts Venue', 'Sake Bar',
       'Art Gallery', 'Arcade', 'Japanese Restaurant',
       'American Restaurant', 'Beer Store', 'Massage Studio', 'BBQ Joint',
       'Bakery', 'Breakfast Spot', 'Latin American Restaurant', 'Bar',
       'Park', 'Bookstore', 'Vegetarian / Vegan Restaurant',
       'Music Venue', 'Cocktail Bar', 'Deli / Bodega', 'Food Truck',
       'Pizza Place', 'Pet Store', 'Sandwich Place', 'Café', 'Pool',
       'Rock Club', 'Wine Bar', 'Health Food Store',
       'Gym / Fitness Center', 'Circus', 'Baseball Stadium',
       'Grocery Store', 'Art Museum', 'Theater', 'Mexican Restaurant',
       'Gastropub', 'Scenic Lookout', 'Plaza', 'Restaurant',
       'Convenience Store', 'Concert Hall', 'Farmers Market',
       'Clothing Store', 'Museum', 'Yoga Studio', 'Fried Chicken Joint',
       'Sports Bar', 'High School', 'Ice Cream Shop',
       'Cajun / Creole Restaurant', 'Jewish Restaurant',
       'Fast Food Restaurant', 'Bur

## Analyze neighborhood

In [140]:
# one hot encoding
la_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

la_onehot.insert(0, 'Neighborhoods', venues_df['Neighborhood'] )

print(la_onehot.shape)
la_onehot.head()

(8170, 366)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cave,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Quad,College Rec Center,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Observatory,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Roman

In [141]:
la_grouped = la_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(la_grouped.shape)
la_grouped.head()

(101, 366)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cave,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Quad,College Rec Center,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Observatory,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Roman

In [144]:
len(la_grouped[la_grouped["Bubble Tea Shop"] > 0])

16

In [146]:
la_boba = la_grouped[["Neighborhoods","Bubble Tea Shop"]]
la_boba.head()

,Neighborhoods,Bubble Tea Shop
0,East Los Angeles,0.00
1,Angeles Mesa,0.00
2,Angelino Heights,0.01
3,Angelus Vista,0.00
4,Arleta,0.00


## Clustering

In [159]:
# set number of clusters
kclusters = 3

la_clustering = la_boba.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0], dtype=int32)

In [160]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
la_merged = la_boba.copy()

# add clustering labels
la_merged["Cluster Labels"] = kmeans.labels_

In [161]:
la_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
la_merged.head()

,Neighborhood,Bubble Tea Shop,Cluster Labels
0,East Los Angeles,0.00,0
1,Angeles Mesa,0.00,0
2,Angelino Heights,0.01,1
3,Angelus Vista,0.00,0
4,Arleta,0.00,0


In [162]:
la_merged = la_merged.join(la_df.set_index("Neighborhood"), on="Neighborhood")

print(la_merged.shape)
la_merged.head() # check the last columns!

(101, 5)


,Neighborhood,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,East Los Angeles,0.00,0,34.033470,-118.159090
1,Angeles Mesa,0.00,0,18.285830,-66.800890
2,Angelino Heights,0.01,1,34.070290,-118.254800
3,Angelus Vista,0.00,0,34.087575,-118.267156
4,Arleta,0.00,0,34.249050,-118.433490


In [163]:
# sort the results by Cluster Labels
print(la_merged.shape)
la_merged.sort_values(["Cluster Labels"], inplace=True)
la_merged

(101, 5)


,Neighborhood,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,East Los Angeles,0.000000,0,34.033470,-118.159090
71,Hancock Park,0.000000,0,34.068920,-118.332550
70,Griffith Park,0.000000,0,34.123270,-118.299900
69,Green Meadows,0.000000,0,33.812272,-118.308367
68,Granada Hills,0.000000,0,34.263940,-118.497940
67,Gramercy Park,0.000000,0,34.033900,-118.312580
66,Glassell Park,0.000000,0,34.119220,-118.230440
65,Garvanza,0.000000,0,34.116970,-118.179990
64,Gallery Row,0.000000,0,34.100800,-118.426140
63,Franklin Hills,0.000000,0,34.103432,-118.349341


In [164]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['Neighborhood'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [165]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [166]:
# list of cluster 0
la_merged.loc[la_merged['Cluster Labels'] == 0]

,Neighborhood,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
0,East Los Angeles,0.0,0,34.033470,-118.159090
71,Hancock Park,0.0,0,34.068920,-118.332550
70,Griffith Park,0.0,0,34.123270,-118.299900
69,Green Meadows,0.0,0,33.812272,-118.308367
68,Granada Hills,0.0,0,34.263940,-118.497940
67,Gramercy Park,0.0,0,34.033900,-118.312580
66,Glassell Park,0.0,0,34.119220,-118.230440
65,Garvanza,0.0,0,34.116970,-118.179990
64,Gallery Row,0.0,0,34.100800,-118.426140
63,Franklin Hills,0.0,0,34.103432,-118.349341


In [167]:
# list of cluster 1
la_merged.loc[la_merged['Cluster Labels'] == 1]

,Neighborhood,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
2,Angelino Heights,0.010000,1,34.070290,-118.254800
8,Baldwin Hills,0.010638,1,34.021570,-118.367650
46,Eagle Rock,0.010000,1,34.139270,-118.210870
80,Hollywood Dell,0.010000,1,34.083570,-118.298597
19,Beverly Park,0.010000,1,34.063720,-118.264600
29,Canoga Park,0.010000,1,34.202390,-118.601560
32,Castle Heights,0.010000,1,34.037999,-118.397516
35,Century City,0.010000,1,34.059440,-118.412810
58,Fairfax,0.010000,1,34.076100,-118.361220
56,Exposition Park,0.010000,1,34.011410,-118.289970


In [168]:
# list of cluster 2
la_merged.loc[la_merged['Cluster Labels'] == 2]

,Neighborhood,Bubble Tea Shop,Cluster Labels,Latitude,Longitude
88,Koreatown,0.03,2,34.057790,-118.300910
30,Canterbury Knolls,0.02,2,34.233482,-118.549533
